In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import glob
import os
from os.path import basename
import numpy as np
import pandas as pd
from PIL import Image
import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,ZeroPadding2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# path="C://rootfop-detection//"
labels= pd.read_csv("labels.csv", delimiter=",",header = None) #load labels


In [24]:
import zipfile
zip_ref = zipfile.ZipFile("images.zip", 'r')
zip_ref.extractall('')
zip_ref.close()

In [25]:
#load rooftop mage and  resize
L=[]
train=np.array([])
images=glob.glob("images"+"/*.*") #importation de images
for i in range(0,len(images)):
        im = Image.open("images//"+labels.iloc[i][0]+".jpg")
        im_rz=im.resize((64,64), Image.ANTIALIAS)
        #im_rz.save(path+"//resized//"+basename(images[i]), 'JPEG',quality=95,optimize=True) #optionally if you want to save rezized images
        L.append(np.array(im_rz))                         
data=np.array(L)


In [26]:
#transformation of labels into categorial variables to be interpreted by keras
y=pd.get_dummies(labels.iloc[:,1])

In [27]:
data

array([[[[ 69,  89,  36],
         [ 71,  93,  41],
         [ 68,  91,  40],
         ...,
         [183, 179, 169],
         [190, 185, 175],
         [198, 194, 183]],

        [[ 59,  79,  29],
         [ 63,  83,  33],
         [ 61,  83,  36],
         ...,
         [170, 166, 155],
         [177, 172, 162],
         [182, 178, 167]],

        [[ 50,  69,  23],
         [ 52,  71,  25],
         [ 55,  76,  31],
         ...,
         [155, 151, 139],
         [159, 155, 143],
         [162, 158, 147]],

        ...,

        [[211, 226, 216],
         [248, 255, 252],
         [246, 255, 252],
         ...,
         [115,  96,  88],
         [137, 120, 112],
         [162, 143, 136]],

        [[171, 188, 178],
         [206, 222, 212],
         [219, 233, 226],
         ...,
         [117,  96,  86],
         [118,  97,  89],
         [120,  96,  89]],

        [[ 60,  77,  67],
         [ 70,  87,  77],
         [ 91, 106,  99],
         ...,
         [143, 118, 110],
        

In [28]:
#seperate data into training set and test set
x_train, x_test, y_train, y_test  = train_test_split(data,y, train_size=0.8)


batch_size = 128
nb_classes = 4
epochs = 180

In [29]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train=np.array(y_train).astype('float32')
y_test=np.array(y_test).astype('float32')
x_train /= 255
x_test /= 255

In [30]:
# start of architecture Convolutional neural network
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(110, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))


model.add(Conv2D(84, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(84, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))


model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))


model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


In [33]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-7)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])  

 # checkpoint: save best model during the training 
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]



#preprocessing and realtime data augmentation:
data_generation = ImageDataGenerator(
    rotation_range=7,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.10,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.10,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images
data_generation.fit(x_train)


In [ ]:
# Fit the model 
model_param=model.fit_generator(data_generation.flow(x_train, y_train,batch_size=batch_size),
                                steps_per_epoch=x_train.shape[0] // batch_size,
                                epochs=epochs,
                                validation_data=(x_test, y_test), callbacks=callbacks_list)

#load best traning model        
model.load_weights(filepath)   

Epoch 1/180
18/18 [==============================] - 96s 5s/step - loss: 1.8832 - acc: 0.2678 - val_loss: 1.3389 - val_acc: 0.2283
Epoch 2/180
18/18 [==============================] - 93s 5s/step - loss: 1.6613 - acc: 0.3234 - val_loss: 1.2731 - val_acc: 0.3667
Epoch 3/180
18/18 [==============================] - 92s 5s/step - loss: 1.3426 - acc: 0.3766 - val_loss: 1.3102 - val_acc: 0.4683
Epoch 4/180
18/18 [==============================] - 94s 5s/step - loss: 1.2899 - acc: 0.4485 - val_loss: 1.2582 - val_acc: 0.4533
Epoch 5/180
18/18 [==============================] - 92s 5s/step - loss: 1.2495 - acc: 0.4185 - val_loss: 1.2968 - val_acc: 0.3700
Epoch 6/180
18/18 [==============================] - 93s 5s/step - loss: 1.2550 - acc: 0.4409 - val_loss: 1.2197 - val_acc: 0.5850
Epoch 7/180
18/18 [==============================] - 92s 5s/step - loss: 1.2437 - acc: 0.4722 - val_loss: 1.3241 - val_acc: 0.3883
Epoch 8/180
18/18 [==============================] - 93s 5s/step - loss: 1.1592 - a

In [ ]:
#show accuracy and confusion matrix
from sklearn.metrics import confusion_matrix

y_pred=np.argmax(model.predict(x_test),1)
cm=confusion_matrix(np.argmax(y_test,1), y_pred)
print("confusion matrix")
print(cm)
print("accuracy")
print(np.sum(np.diag(cm))/np.sum(cm)) 
